Packages

In [34]:
import os
import numpy
import pandas
import tqdm
from pyspark.sql import SparkSession
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline
import sys
sys.path.append('../scripts/')
from read_utils import read_file, create_folder, temp_record_query, temp_record_sdf

In [3]:
spark = (
    # Create a spark session (which will run spark jobs)
    SparkSession.builder.appName("Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config('spark.executor.memory','10g')
    .config('spark.driver.memory','12g')
    .config('spark.driver.maxResultsSize', '10GiB')
    # .config("spark.network.timeout", "3600s")
    # .master("local[6]")
    .getOrCreate()
    )

22/09/21 00:23:54 WARN Utils: Your hostname, Runyus-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.3.12 instead (on interface en0)
22/09/21 00:23:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/21 00:23:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [54]:
file_name_list = sorted(os.listdir('../data/curated/data'))

- Bugs(can not read all data)

In [64]:
transactions_first_df = read_file(spark, ptw ='../data/curated/data/', file_name=file_name_list[0])
for i in range(len(file_name_list)):
    sub_df = read_file(spark, ptw = '../data/curated/data/', file_name = file_name_list[i])
    new_df = transactions_first_df.union(sub_df)

|> Loading File...
|> Loading Finished!
-RECORD 0-----------------------------------------------
 order_id        | 0068ea48-d837-4293-8c46-02fa8012009b 
 user_id         | 23675                                
 merchant_abn    | 76070803743                          
 dollar_value    | 102.37539050484392                   
 merchant_name   | Massa Corp.                          
 tags            | shoe shops                           
 take_rate       | 6.19                                 
 type            | a                                    
 consumer_id     | 236013                               
 name            | Susan Woods                          
 address         | 75974 William Lodge                  
 state           | SA                                   
 postcode        | 5007                                 
 gender          | Female                               
 AUS             | -26                                  
 NSW             | -48                          

In [66]:
tag_indexer = StringIndexer(inputCol = "tags",outputCol = "tagIndex", handleInvalid="skip")

type_indexer = StringIndexer(inputCol = "type", outputCol = "typeIndex", handleInvalid="skip")

onehotencoder_tag_vector = OneHotEncoder(inputCol = "tagIndex", outputCol = "tag_vec")
#Create pipeline and pass all stages
pipeline = Pipeline(stages=[tag_indexer,
                            type_indexer,
                            onehotencoder_tag_vector])

In [67]:
df_transformed = pipeline.fit(new_df).transform(new_df)
df_transformed.show()

+--------------------+-------+------------+------------------+--------------------+--------------------+---------+----+-----------+--------------------+--------------------+-----+--------+-----------+---+---+---+---+---+---+---+---+---+------+---------------+--------+---------+---------------+
|            order_id|user_id|merchant_abn|      dollar_value|       merchant_name|                tags|take_rate|type|consumer_id|                name|             address|state|postcode|     gender|AUS|NSW|VIC|QLD| SA| WA|TAS| NT|ACT|target|New cases / day|tagIndex|typeIndex|        tag_vec|
+--------------------+-------+------------+------------------+--------------------+--------------------+---------+----+-----------+--------------------+--------------------+-----+--------+-----------+---+---+---+---+---+---+---+---+---+------+---------------+--------+---------+---------------+
|0068ea48-d837-429...|  23675| 76070803743|102.37539050484392|         Massa Corp.|          shoe shops|     6.19| 